In [ ]:
%pip install pandas

import pandas as pd

In [22]:
movies = pd.read_csv('../data/TMDB_all_movies.csv')

movies = movies[['id', 'title', 'overview', 'genres', 'release_date']]
horror_movies = movies.loc[(movies['genres'] == 'Horror')]

horror_movies['tags'] = horror_movies['overview']+movies['genres']
horror_movies = horror_movies.drop(columns=['overview', 'genres'])
horror_movies.dropna()

/var/folders/j8/rb8khncs0j73tq5y5x9g6d980000gn/T/ipykernel_24484/4272560742.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  horror_movies['tags'] = horror_movies['overview']+movies['genres']


,id,title,release_date,tags
100,138,Dracula,1931-02-12,British estate agent Renfield travels to Trans...
170,215,Saw II,2005-10-28,The chilling and relentless Jigsaw killer retu...
297,377,A Nightmare on Elm Street,1984-10-31,Teenagers in a small town are dropping like fl...
483,609,Poltergeist,1982-06-04,Upon realizing that something truly evil haunt...
524,653,Nosferatu,1922-02-16,The mysterious Count Orlok summons Thomas Hutt...
...,...,...,...,...
971714,1333609,Nona,2021-12-12,A young man in prison recounts the events that...
971915,1333810,Eve Was Star,2024-05-01,A young man on the run from his past meets a g...
973923,1335865,Midwest Angelica Interlude A,2024-02-06,"In the year 1999, a corpse descends from the c..."
973945,1335887,The Dare,2021-12-02,A dare gone wrong leads to an unexpected encou...


In [ ]:
horror_movies.count()
# horror_movies.columns

In [ ]:
%pip install scikit-learn nltk

In [ ]:
# Text cleaning
import nltk
import re
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize

nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('wordnet')
nltk.download('stopwords')

def clean_text(text):
    # Check if text is not a string
    if not isinstance(text, str):
        return ""

    # Convert to lowercase
    text = text.lower()

    # Remove punctuation but keep digits
    text = re.sub(r'[^\w\s\d]', '', text)

    # Tokenize
    words = word_tokenize(text)

    # Remove stop words
    stop_words = set(stopwords.words('english'))
    words = [word for word in words if word not in stop_words]

    # Lemmatize
    lemmatizer = WordNetLemmatizer()
    words = [lemmatizer.lemmatize(word) for word in words]

    # Join the words back together
    text = ' '.join(words)
    return text

In [ ]:
horror_movies['tags_clean'] = horror_movies['tags'].apply(clean_text)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split

train_data, test_data = train_test_split(horror_movies, test_size=0.2, random_state=42)

cv = CountVectorizer(max_features=25000, stop_words='english')

In [ ]:
vector = cv.fit_transform(horror_movies['tags_clean'].values.astype('U')).toarray()

In [ ]:
vector.shape

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

similarity = cosine_similarity(vector)

In [ ]:
distance = sorted(list(enumerate(similarity[0])), reverse=True, key=lambda vector:vector[1])
for i in distance[0:5]:
  print(f"{horror_movies.iloc[i[0]].title} ({horror_movies.iloc[i[0]].release_date.split('-')[0]})")

In [ ]:
def recommend(movies):
  try:
    index = horror_movies[horror_movies['title'] == movies].index[0]
    distance = sorted(list(enumerate(similarity[index])), reverse=True, key=lambda vector:vector[1])
    for i in distance[1:6]:
      print(f"{horror_movies.iloc[i[0]].title} ({horror_movies.iloc[i[0]].release_date.split('-')[0]})")
  except:
    print("Doesn't exist")

In [19]:
recommend('Poltergeist')

Lost Command (2012)
Dead Body Man Chronicles (2008)
Dead Island: No Retreat (2013)
Still Alive (2015)
Reich of the Reich (2022)
